In [ ]:
# Setup session and imports
import streamlit as st
import pandas as pd
import json

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Set context
session.sql("USE DATABASE CALL_CENTER_ANALYTICS").collect()
session.sql("USE SCHEMA AUDIO_PROCESSING").collect()
session.sql("USE WAREHOUSE AUDIO_CORTEX_WH").collect()

st.write("✅ AI Analytics session ready - Let's compete with GONG!")


In [ ]:
-- Generate executive summaries for all call transcripts
CREATE OR REPLACE TABLE CALL_SUMMARIES AS
SELECT 
    audio_file_name,
    transcript_text,
    SNOWFLAKE.CORTEX.SUMMARIZE(transcript_text) AS executive_summary,
    processing_timestamp
FROM CALL_TRANSCRIPTS;

-- Display the results
SELECT 
    audio_file_name,
    LEFT(transcript_text, 100) || '...' AS transcript_preview,
    executive_summary
FROM CALL_SUMMARIES;


In [ ]:
-- Add sentiment analysis to our call data
CREATE OR REPLACE TABLE CALL_SENTIMENT_ANALYSIS AS
SELECT 
    cs.*,
    SNOWFLAKE.CORTEX.SENTIMENT(transcript_text) AS sentiment_score,
    CASE 
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript_text) >= 0.1 THEN '😊 Positive'
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript_text) <= -0.1 THEN '😞 Negative'
        ELSE '😐 Neutral'
    END AS sentiment_category
FROM CALL_SUMMARIES cs;

-- Display sentiment analysis results
SELECT 
    audio_file_name,
    sentiment_score,
    sentiment_category,
    executive_summary
FROM CALL_SENTIMENT_ANALYSIS
ORDER BY sentiment_score DESC;


In [ ]:
-- Classify calls into business categories
CREATE OR REPLACE TABLE CALL_CLASSIFICATION AS
SELECT 
    csa.*,
    AI_CLASSIFY(
        transcript_text, 
        ['Complaint', 'Inquiry', 'Compliment', 'Technical Support', 'Billing Issue', 'General Information']
    ) AS call_classification
FROM CALL_SENTIMENT_ANALYSIS csa;

-- Display classification results
SELECT 
    audio_file_name,
    call_classification,
    sentiment_category,
    LEFT(executive_summary, 100) || '...' AS summary_preview
FROM CALL_CLASSIFICATION;


In [ ]:
-- Add intelligent filtering for escalation and priority identification
CREATE OR REPLACE TABLE CALL_PRIORITY_ANALYSIS AS
SELECT 
    cc.*,
    AI_FILTER(PROMPT('Does this call indicate the request needs escalating? {0}', transcript_text)) AS needs_escalation,
    AI_FILTER(PROMPT('Is this caller expressing dissatisfaction or frustration? {0}', transcript_text)) AS high_priority_call
FROM CALL_CLASSIFICATION cc;

-- Display priority analysis results
SELECT 
    audio_file_name,
    call_classification,
    sentiment_category,
    needs_escalation,
    high_priority_call,
    CASE 
        WHEN needs_escalation AND high_priority_call THEN '🔴 URGENT'
        WHEN needs_escalation OR high_priority_call THEN '🟡 PRIORITY'
        ELSE '🟢 STANDARD'
    END AS priority_level
FROM CALL_PRIORITY_ANALYSIS;


In [ ]:
-- Extract comprehensive call intelligence using AI_COMPLETE with structured output
CREATE OR REPLACE TABLE COMPREHENSIVE_CALL_INTELLIGENCE AS
SELECT 
    cpa.audio_file_name,
    cpa.transcript_text,
    cpa.executive_summary,
    cpa.sentiment_score,
    cpa.call_classification,
    AI_COMPLETE(
        model => 'snowflake-arctic',
        prompt => 'Extract comprehensive call analytics from this call center transcript. Focus on actionable business insights that would compete with GONG platform capabilities: ' || cpa.transcript_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'call_summary': {'type': 'string', 'description': 'Brief overview of the entire conversation'},
                    'key_topics_discussed': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Main topics covered in the call'},
                    'customer_sentiment': {'type': 'string', 'enum': ['very_positive', 'positive', 'neutral', 'negative', 'very_negative']},
                    'agent_performance': {'type': 'string', 'enum': ['excellent', 'good', 'satisfactory', 'needs_improvement', 'poor']},
                    'issue_resolved': {'type': 'string', 'enum': ['fully_resolved', 'partially_resolved', 'unresolved', 'follow_up_scheduled']},
                    'next_steps': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Specific action items identified'},
                    'follow_up_required': {'type': 'boolean'},
                    'escalation_needed': {'type': 'boolean'},
                    'customer_satisfaction_indicators': {'type': 'array', 'items': {'type': 'string'}},
                    'call_outcome': {'type': 'string', 'enum': ['successful', 'unsuccessful', 'pending_resolution']},
                    'action_items_for_agent': {'type': 'array', 'items': {'type': 'string'}},
                    'business_impact': {'type': 'string', 'enum': ['high', 'medium', 'low']},
                    'competitive_mentions': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Any competitor mentions'},
                    'upsell_opportunities': {'type': 'array', 'items': {'type': 'string'}}
                },
                'required': ['call_summary', 'key_topics_discussed', 'customer_sentiment', 'agent_performance', 'issue_resolved', 'next_steps', 'follow_up_required', 'escalation_needed']
            }
        }
    ) AS comprehensive_intelligence
FROM CALL_PRIORITY_ANALYSIS cpa;

-- Display the comprehensive intelligence results
SELECT 
    audio_file_name,
    comprehensive_intelligence
FROM COMPREHENSIVE_CALL_INTELLIGENCE;
